# 2 Data wrangling

## 2.1 Contents

## 2.2 Introduction

Data wrangling is the process of cleaning, transforming, and organizing raw data into a usable format. Below are the steps for performing data wrangling on the "Amazon Fine Food Reviews" dataset with the columns Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, 

### 2.2.1 Data Science Problem

The "Amazon Fine Food Reviews" dataset offers valuable insights into customer sentiment and preferences through product reviews. By applying natural language processing (NLP) and machine learning techniques, the challenge is to analyze review texts, ratings, and other attributes to identify patterns and trends in customer feedback. This will help uncover key themes related to product quality, delivery experience, and overall satisfaction. The goal is to provide actionable insights for improving product offerings, enhancing customer service, and informing marketing strategies, ultimately leading to better customer understanding and increased loyalty.

## 2.3 Imports

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3
import nltk

## 2.4 Load The Data

In [2]:
con = sqlite3.connect('database.sqlite')

In [3]:
amazon_data = pd.read_sql_query(""" SELECT * FROM Reviews""", con)
amazon_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## 2.5 Handling Missing Values

In [4]:
# Check for missing values
missing_values = amazon_data.isnull().sum()
print(missing_values)

# Example: Fill missing values in 'ProfileName' with 'Unknown'
amazon_data['ProfileName'].fillna('Unknown', inplace=True)

# Drop rows where 'Text' is missing, as text data is crucial
amazon_data.dropna(subset=['Text'], inplace=True)

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64


There are no missing values in the dataset.

## 2.6 Duplicate Values

### 2.6.1 Duplicate records

In [5]:
# List of columns to check for duplicates, excluding 'Id'
columns_to_check = [col for col in amazon_data.columns if col != 'Id']

# Check for duplicate records based on all columns except 'Id'
duplicate_records = amazon_data[amazon_data.duplicated(subset=columns_to_check, keep=False)]

In [6]:
duplicate_records.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
6516,6517,B005O8BLLU,APH7I7OZ8WUJP,J. Simpson,0,0,5,1347494400,Great first food,This is excellent for a baby's first taste. Th...
6517,6518,B005O8BLLU,APH7I7OZ8WUJP,J. Simpson,0,0,5,1347494400,Great first food,This is excellent for a baby's first taste. Th...
8522,8523,B003VXFK44,A10H24TDLK2VDP,William Jens Jensen,0,0,3,1309824000,Unremarkable,"First, let me say that I prefer extra-bold K-C..."
8523,8524,B003VXFK44,A10H24TDLK2VDP,William Jens Jensen,0,0,3,1309824000,Unremarkable,"First, let me say that I prefer extra-bold K-C..."
9231,9232,B006N3IG4K,A10H24TDLK2VDP,William Jens Jensen,0,0,3,1309824000,Unremarkable,"First, let me say that I prefer extra-bold K-C..."


In [7]:
duplicate_records.shape

(506, 10)

There are a total of 506 duplicate records identified in the dataset.

### 2.6.1 Duplicate reviews

In [8]:
# Store non-duplicate records
non_duplicate_records = amazon_data.drop_duplicates(keep=False)

# Check for duplicate records based on ProductId, UserId, and ProfileName
duplicate_reviews = non_duplicate_records[non_duplicate_records.duplicated(subset=['ProductId', 'UserId', 'ProfileName', 'Time'], keep=False)]

In [9]:
duplicate_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
425,426,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,0,0,4,1334016000,glad to find them in 1 oz size,"I buy mostly for vending, so the size of the b..."
426,427,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,0,0,4,1334016000,"pretty good, could be better",Glad to find these in a one ounce size but the...
434,435,B000G6RYNE,A15USNEAJUXOSH,L. Schrank,0,0,4,1326067200,Quite good,I enjoy these chips. I got these instead of my...
435,436,B000G6RYNE,A15USNEAJUXOSH,L. Schrank,0,0,5,1326067200,Delicious,"I love these chips, I buy the 24 pack once a m..."
462,463,B000G6RYNE,A3RMGIKUWGPZOK,Jean Visnefski,0,0,5,1245024000,One of Their Best Flavors,Kettle Chips flavors can be hit or miss. Some...


In [10]:
duplicate_reviews.shape

(7424, 10)

There are 7,424 duplicate reviews in the dataset for the same product (identified by ProductId), submitted by the same user (identified by UserId) at the same time (identified by Time).

## 2.6 Creating a Helpfulness Ratio

In [11]:
# Create a new column for the helpfulness ratio
amazon_data['HelpfulnessRatio'] = amazon_data['HelpfulnessNumerator'] / amazon_data['HelpfulnessDenominator']

# Handle division by zero and missing values by filling with 0
amazon_data['HelpfulnessRatio'].fillna(0, inplace=True)

In [12]:
amazon_data['HelpfulnessRatio'].value_counts(ascending=False)

0.000000    303826
1.000000    183309
0.500000     21623
0.666667     10514
0.750000      6364
             ...  
0.946237         1
0.232558         1
0.704545         1
0.161290         1
0.991411         1
Name: HelpfulnessRatio, Length: 951, dtype: int64

The output shows the distribution of the HelpfulnessRatio in the amazon_data DataFrame. The data reveals that a significant number of reviews are marked as either completely unhelpful (0.0) or entirely helpful (1.0), with counts of 303,826 and 183,309 respectively. Intermediate values, such as 0.5, 0.67, and 0.75, appear less frequently, indicating some reviews receive mixed feedback. 

## 2.7 Converting Time to DateTime

In [13]:
# Convert Time column to datetime
amazon_data['ReviewTime'] = pd.to_datetime(amazon_data['Time'], unit='s')

In [14]:
amazon_data['ReviewTime'].value_counts(ascending=False)

2012-10-16    1143
2011-11-25    1088
2011-11-28    1070
2012-09-06    1018
2012-08-06     989
              ... 
2004-09-20       1
2004-12-10       1
2004-03-26       1
2004-06-20       1
2003-11-25       1
Name: ReviewTime, Length: 3168, dtype: int64

October 16, 2012, has the highest frequency of customer reviews.

In [15]:
print('The earliest review times', amazon_data['ReviewTime'].min(), 'and the latest review times', amazon_data['ReviewTime'].max())

The earliest review times 1999-10-08 00:00:00 and the latest review times 2012-10-26 00:00:00


## 2.8 Text Preprocessing

In [16]:
import re

# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Apply the clean_text function to the Text and Summary columns
amazon_data['CleanedText'] = amazon_data['Text'].apply(clean_text)
amazon_data['CleanedSummary'] = amazon_data['Summary'].apply(clean_text)

The preprocessing of text data converts the 'Text' and 'Summary' columns in the amazon_data dataframe to lowercase, and removes digits, punctuation, extra spaces, and leading/trailing spaces to create cleaned versions of these columns.

## 2.9 Tokenization and Stopword Removal

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure you have the stopwords and punkt data downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

# Function to tokenize and remove stopwords
def tokenize_text(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply the tokenize_text function to the CleanedText and CleanedSummary columns
amazon_data['TokenizedText'] = amazon_data['CleanedText'].apply(tokenize_text)
amazon_data['TokenizedSummary'] = amazon_data['CleanedSummary'].apply(tokenize_text)

he code tokenizes the cleaned text data from the 'CleanedText' and 'CleanedSummary' columns in the amazon_data dataframe and removes stopwords, creating tokenized versions of these columns.

## 2.10 Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Function to lemmatize tokens
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply the lemmatize_tokens function to the TokenizedText and TokenizedSummary columns
amazon_data['LemmatizedText'] = amazon_data['TokenizedText'].apply(lemmatize_tokens)
amazon_data['LemmatizedSummary'] = amazon_data['TokenizedSummary'].apply(lemmatize_tokens)

The code lemmatizes the tokenized text data from the 'TokenizedText' and 'TokenizedSummary' columns in the amazon_data dataframe, creating lemmatized versions of these columns to ensure words are reduced to their base or root form.

## 2.11 Saving the Cleaned Data

In [21]:
amazon_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,HelpfulnessRatio,ReviewTime,CleanedText,CleanedSummary,TokenizedText,TokenizedSummary,LemmatizedText,LemmatizedSummary
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1.0,2011-04-27,i have bought several of the vitality canned d...,good quality dog food,"[bought, several, vitality, canned, dog, food,...","[good, quality, dog, food]","[bought, several, vitality, canned, dog, food,...","[good, quality, dog, food]"
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0.0,2012-09-07,product arrived labeled as jumbo salted peanut...,not as advertised,"[product, arrived, labeled, jumbo, salted, pea...",[advertised],"[product, arrived, labeled, jumbo, salted, pea...",[advertised]
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1.0,2008-08-18,this is a confection that has been around a fe...,delight says it all,"[confection, around, centuries, light, pillowy...","[delight, says]","[confection, around, century, light, pillowy, ...","[delight, say]"
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,1.0,2011-06-13,if you are looking for the secret ingredient i...,cough medicine,"[looking, secret, ingredient, robitussin, beli...","[cough, medicine]","[looking, secret, ingredient, robitussin, beli...","[cough, medicine]"
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0.0,2012-10-21,great taffy at a great price there was a wide ...,great taffy,"[great, taffy, great, price, wide, assortment,...","[great, taffy]","[great, taffy, great, price, wide, assortment,...","[great, taffy]"


In [20]:
# Save the cleaned data to a new CSV file
amazon_data.to_csv('amazon_data_cleaned.csv', index=False)

## 2.12 Summary

The above steps provide a comprehensive guide for wrangling the amazon_data DataFrame. The process includes handling missing values, creating new features, preprocessing text data, tokenizing and lemmatizing, encoding categorical variables, and saving the cleaned data. This ensures the dataset is in a usable format for further analysis and modeling tasks.